In [383]:
import pandas as pd
from lib import standardize_item_numbers
import numpy as np

In [384]:
def clean():
    dfa = pd.read_csv("../data/police_reports/electronic_police_report_2018_2022.csv")
    dfa = dfa.pipe(standardize_item_numbers, ["item_number"])

    dfb = pd.read_csv("../data/real_time_crime_center/rtcc.csv")
    dfb["rtcc_requested"] = "1"
    dfb = dfb.pipe(standardize_item_numbers, ["item_number"])

    df = pd.merge(dfa, dfb, on="item_number")
    return df

In [385]:
df = clean()

C:\Users\PC\AppData\Local\Temp\ipykernel_2456\153542535.py:2: DtypeWarning: Columns (14,20,24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  dfa = pd.read_csv("../data/police_reports/electronic_police_report_2018_2022.csv")


In [391]:
df.loc[:, "location"] = df.location.str.replace(r"(\w+)$", r"\1, New Orleans, Louisiana")

requested = df[["item_number", "location"]]

requested_1 = requested[:5000]
requested_2 = requested[5000:10000]
requested_3 = requested[10000:15000]
requested_4 = requested[15000:]

# requested_1.to_csv("../data/real_time_crime_center/rtcc_locations_1.csv", index=False)
# requested_2.to_csv("../data/real_time_crime_center/rtcc_locations_2.csv", index=False)
# requested_3.to_csv("../data/real_time_crime_center/rtcc_locations_3.csv", index=False)
# requested_4.to_csv("../data/real_time_crime_center/rtcc_locations_4.csv", index=False)

C:\Users\PC\AppData\Local\Temp\ipykernel_2456\1998154791.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df.loc[:, "location"] = df.location.str.replace(r"(\w+)$", r"\1, New Orleans, Louisiana")


In [392]:
requested_1_coordinates = pd.read_csv("../data/real_time_crime_center/rtcc_locations_1_coordinates.csv")
requested_1_df = pd.read_csv("../data/real_time_crime_center/rtcc_locations_1.csv")
rtcc_1 = pd.concat([requested_1_df, requested_1_coordinates], axis=1)

In [393]:
requested_2_coordinates_df = pd.read_csv("../data/real_time_crime_center/rtcc_locations_2_coordinates.csv")
requested_2_df = pd.read_csv("../data/real_time_crime_center/rtcc_locations_2.csv")
rtcc_2 = pd.concat([requested_2_df, requested_2_coordinates_df], axis=1)

In [394]:
requested_3_coordinates_df = pd.read_csv("../data/real_time_crime_center/rtcc_locations_3_coordinates.csv")
requested_3_df = pd.read_csv("../data/real_time_crime_center/rtcc_locations_3.csv")
rtcc_3 = pd.concat([requested_3_df, requested_3_coordinates_df], axis=1)

In [395]:
requested_4_coordinates_df = pd.read_csv("../data/real_time_crime_center/rtcc_locations_4_coordinates.csv")
requested_4_df = pd.read_csv("../data/real_time_crime_center/rtcc_locations_4.csv")
rtcc_4 = pd.concat([requested_4_df, requested_4_coordinates_df], axis=1)

In [396]:
coordinates_df = pd.concat([rtcc_1, rtcc_2, rtcc_3, rtcc_4], axis=0)

In [397]:
# coordinates_df.to_csv("../data/real_time_crime_center/rtcc_coordinates.csv", index=False)

In [399]:
coordinates_df = pd.read_csv("../data/real_time_crime_center/rtcc_coordinates.csv")
coordinates_df = coordinates_df.rename(columns={"0": "latitude", "1": "longitude"})
coordinates_df = coordinates_df[["item_number", "latitude", "longitude"]]

In [400]:
df = pd.merge(df, coordinates_df, on="item_number")

In [401]:
df = df.drop_duplicates()

In [402]:
df['occurred_date_time'] = sorted(pd.to_datetime(df['occurred_date_time']))
df['hour'] = df['occurred_date_time'].dt.hour
df = df[~((df.hour.fillna("") == ""))]
df.loc[:, "hour"] = df.hour.astype(str)

night = df[df.hour.isin(["19", "20", "21", "22", "23", "00", "1", "2", "3", "4", "5", "6"])]
night = night[["latitude", "longitude", "item_number"]]

day = df[~((df.hour.isin(["19", "20", "21", "22", "23", "00", "1", "2", "3", "4", "5", "6"])))]
day = day[["latitude", "longitude", "item_number"]]


# night.to_csv("../data/real_time_crime_center/rtcc_requested_locations_night.csv")
# day.to_csv("../data/real_time_crime_center/rtcc_requested_locations_day.csv")